Original code by Dede et al. (https://figshare.com/articles/software/enCas12a_screen_analysis_pipeline/12275642), licensed under CC BY 4.0
https://creativecommons.org/licenses/by/4.0/
Modified by Hamda Ajmal, March 2025

Changes: Extracted relevant portions of the code and applied them to different datasets.

In [ ]:

## What am I doing here? I am going to change Parrish plasmids to ETP.

%matplotlib inline
%pylab inline
import pandas as pd
import scipy.stats as stats
import scipy.cluster.hierarchy as clust
import seaborn as sns
import matplotlib.pyplot as plt
reads = pd.read_csv('../InputData/Parrish\GSE178179_pgPEN_counts_PC9.txt', index_col=0, sep='\t')


In [ ]:
reads.columns

In [ ]:
reads.drop(['gRNA1_seq', 'gRNA2_seq', 'PC9_plasmid', 'PC9_LTP_RepA', 'PC9_LTP_RepB', 'PC9_LTP_RepC',
           'PC9_ETP_RepA', 'PC9_ETP_RepB', 'PC9_ETP_RepC', 'HeLa_plasmid'], axis=1, inplace=True)
reads


In [ ]:
numGuides, numSamples = reads.shape

numGuides, numSamples 
reads

In [ ]:
sample_sum = reads.iloc[:,range(1,numSamples)].sum(0)
highlight_index = 0
colors = ['tab:blue'] * len(sample_sum)
colors[highlight_index] = 'tab:red'
figure( figsize(6,4))
barh( arange(len(sample_sum)), sample_sum, align='center', color = colors)
#plot( [-0.5, len(sample_sum)-0.5], [numGuides*500, numGuides*500], 'r--')  #This is for the red dashed line, r for red
#Format string fmt = '[marker][line][color]' why is this line

ylim(-1, len(sample_sum)) # len(sample_sum) = 10
yticks(arange(len(sample_sum)), reads.columns.values[1:], rotation=0)

show()


In [ ]:
pseudo=5

reads[ reads.columns[1:]] = reads[ reads.columns[1:]] + pseudo
#reads.head()

In [ ]:
meanReads = reads.iloc[:,range(1,numSamples)].mean(0) # colmean
meanReads

In [ ]:
# also remove double control "pgRNAs. Five hundred double non-targeting pgRNAs were included as a control"
filtered_reads = reads[reads['paralog_pair'].str.contains('^NTpg.*\|NA$')].index
filtered_reads.shape
filtered_reads

In [ ]:
#reads['paralog_pair'].str.contains('^NTpg.*\|NA$')
reads.drop(filtered_reads, axis = 0, inplace = True)
reads

In [ ]:
numGuides, numSamples = reads.shape

numGuides, numSamples 

In [ ]:
normed = pd.DataFrame(index=reads.index, columns=reads.columns) # empty data frame
normed['paralog_pair'] = reads.iloc[:, 0]
normed.rename(columns={"paralog_pair": "GENE"}, inplace = True)
normed
#pseudo has already been added to reads
#The numpy.tile() function constructs a new array by repeating array – ‘arr’, 
#the number of times we want to repeat as per repetitions. 
#The resulted array will have dimensions max(arr.ndim, repetitions) where, 
#repetitions is the length of repetitions. If arr.ndim > repetitions,
#reps is promoted to arr.ndim by pre-pending 1’s to it.
#If arr.ndim < repetitions, reps is promoted to arr.ndim by pre-pending new axis. Syntax : 
normed[ normed.columns[1:] ] =   (reads.iloc[:, range(1, numSamples)] ) / np.tile(meanReads.values, [numGuides, 1]) * 500  # normalize to mean 500 read
normed.head()


In [ ]:
fc = pd.DataFrame(index=reads.index, columns=reads.columns[[2,3,4]]) # non-normalised
fc['GENE'] = reads['paralog_pair']  # first column is gene name
fc
#fc.shape # share is for dimensions - nice
numFCsamples = fc.shape[1]-1   # number of columns for which to calculate FC
numFCsamples
#fc.head()
pseudo = 0 # remmeber pseudocount is already in the data 
fc.head()
## rearrane here 
cols = ['GENE' ,'HeLa_LTP_RepA' ,'HeLa_LTP_RepB' ,'HeLa_LTP_RepC']
fc = fc[cols]
fc

## VERY IMPORTANT TO REARRANGE NORMED HERE
cols = ['GENE', 'HeLa_LTP_RepA', 'HeLa_LTP_RepB', 'HeLa_LTP_RepC', 'HeLa_ETP']
normed = normed[cols]

fc[ fc.columns[1:] ] = log2( (normed[ normed.columns[1:-1]] + pseudo) / np.tile( normed[ normed.columns[-1]] + pseudo , [numFCsamples, 1]).T )
fc

In [ ]:
## Anything with nt  is control in this study
fc_base = pd.DataFrame(index=fc.index, columns=fc.columns) # non-normalised
fc_base.iloc[:] = fc.iloc[:]
fc_base
fc_base[['GENE1', 'GENE2']] = fc_base.GENE.str.split("|", expand = True)
fc_base.drop(['GENE'], inplace = True,axis=1)
fc_base
fc_base.loc[fc_base['GENE1'].str.contains('^nt[0-8]{1}'),'GENE1'] = 'control'
fc_base.loc[fc_base['GENE2'].str.contains('^nt[0-8]{1}'),'GENE2'] = 'control'

fc_base

In [ ]:
cells = list([ 'HeLa'])
cols = list(['GENE1']) + list( ['GENE2']) + cells
cols
fc_merge = pd.DataFrame( columns=cols, index=fc_base.index, dtype=float)
fc_merge.GENE1 = fc_base.GENE1
fc_merge.GENE2 = fc_base.GENE2

for cell in cells:
    samples = [x for x in fc_base.columns if cell in x]
    fc_merge[cell] = fc[ samples ].mean(1)

fc_merge

In [ ]:
is_ctrl = where( (fc_merge.GENE1=='control') | (fc_merge.GENE2=='control') )[0]

is_ctrl1 = where( fc_merge.GENE1=='control' )[0]
is_ctrl2 = where( fc_merge.GENE2=='control' )[0]


smf_gene1 = fc_merge.iloc[is_ctrl2].groupby('GENE1').mean(numeric_only = True) # calculate means of multiple runs of same gene
smf_gene2 = fc_merge.iloc[is_ctrl1].groupby('GENE2').mean(numeric_only = True)


smf_guide1 = fc_merge.iloc[is_ctrl2].groupby('GENE1')
smf_guide2 = fc_merge.iloc[is_ctrl1].groupby('GENE2')

In [ ]:
smf_gene = pd.concat([smf_gene1, smf_gene2], ignore_index=False)#smf_gene = smf_gene1.join(smf_gene2, lsuffix='_Aposn', rsuffix='_Bposn')
smf_gene# mean of same gene-control pair has already been calculated

In [ ]:
pairs = fc_merge[(fc_merge['GENE1'] != "control") & (fc_merge['GENE2'] != "control")] # These are all experiments without control,
pairs =  pairs[["GENE1", "GENE2"]]
pairs =pairs.drop_duplicates(keep='first')
pairs.insert(2, "GENE1_GENE2",np.tile("ZZ",len(pairs)), True)
pairs
pairs.columns
for ind in pairs.index:
    g1 = pairs.loc[ind, 'GENE1']
    g2 = pairs.loc[ind, 'GENE2']
    newval = g1 + "_" + g2
    if g1 > g2:
        newval = g2 + "_" + g1
    pairs.loc[ind,"GENE1_GENE2"] = newval
#    print(g1_g2)
#print(pairs.columns)
   
pairs
pairs.drop_duplicates(subset="GENE1_GENE2", keep="first",inplace = True)
print(len(pairs)) # From Paper: we here report our direct experimental evaluation of GIs among 1,030 paralog pairs (2,060 genes) in two human cell contexts.

In [ ]:
dLFC = pd.DataFrame( index=list(pairs.GENE1 + "_" + pairs.GENE2), columns=fc_merge.columns[2:], dtype=float)
dLFC

In [ ]:
smf = smf_gene
for pair_idx in pairs.index:
    g1 = pairs.loc[pair_idx].GENE1
    g2 = pairs.loc[pair_idx].GENE2
    expt_idx  = list( where( ( (fc_merge.GENE1==g1) & (fc_merge.GENE2==g2) )  | ( (fc_merge.GENE1==g2) & (fc_merge.GENE2==g1)  ))[0] )
    
    if ( len(expt_idx)==0 ):
        continue
    smf_sum = smf.loc[g1] + smf.loc[g2]
    expt = fc_merge.iloc[ expt_idx ]
    genepair = g1 + "_" + g2
    dLFC.loc[genepair] = expt.median(0,numeric_only=True) - smf_sum

In [ ]:
zdLFC = pd.DataFrame( index=dLFC.index, columns=dLFC.columns, dtype=float ) 
zdLFC.head()
percentile = 2.5
for col in zdLFC.columns:
    #print(col)
    top = np.percentile( dLFC.loc[:,col], percentile)
    bot = np.percentile( dLFC.loc[:,col], 100-percentile)
    #print(top,bot)
    mu = dLFC.iloc[ where( (dLFC[col]>top) & (dLFC[col]<bot))[0] ][col].mean() # This is because our dist is truncated normal
    std = dLFC.iloc[ where( (dLFC[col]>top) & (dLFC[col]<bot))[0] ][col].std()
    #print(mu, std)
    zdLFC[col] = (dLFC[col] - mu) / std

In [ ]:
xx = linspace(-10,10,500)
kde_HeLa = stats.gaussian_kde( zdLFC.HeLa )
figure( figsize(5,4) )
plot( xx, stats.norm.pdf( xx), label='normal', linewidth=4 )
plot( xx, kde_HeLa.evaluate(xx), label='HeLa')

legend(loc=2)
#savefig('normfit-of-zdLFC.pdf')
show()


In [ ]:
def reindex_alphbetically(df):
    result = []
    for index, row in df.iterrows():
        a, b = index.split('_')
        if a < b:
            result.append(f'{a}_{b}')
        else:
            result.append(f'{b}_{a}')
    
    
    return(result)


zdLFC.index = reindex_alphbetically(zdLFC)
zdLFC.to_csv("zdLFC Output/Parrish_Hela.csv", index=True)  # Set index=False to exclude the index column